In [ ]:
import math
import pandas as pd
import seaborn as sns 
import tellurium as te
import matplotlib.pyplot as plt

fn_ant = '../model/PBK PFAS generic.ant'

generic_PFAS_model = te.loada(fn_ant)

# Helper function for plotting simulation results
def plotResults(results,plot_params):
    sim_result = pd.DataFrame(results, columns = results.colnames)
    df_sim = pd.melt(sim_result, id_vars=['age'], value_vars=plot_params, var_name='variable', value_name='value')
    g = sns.FacetGrid(df_sim, col='variable', col_wrap=5, sharey=False, col_order=plot_params)
    g.map(sns.lineplot, 'age', 'value')
    g.add_legend()

def loadParametrisation(model, filename):
    df = pd.read_csv(filename)
    df['Value'] = df['Value'].astype(float)
    for index, row in df.iterrows():
        model[str(row['Parameter'])] = row['Value']
        
# Set chemical parameters
loadParametrisation(rr_model, '../Parametrisations/PFOA_Human.csv')

# Simulate daily oral and dermal dosing via an event
model = generic_PFAS_model

# Remove all events
for event_id in model.model.getEventIds():
    model.removeEvent(event_id)

# Make sure the compartment is not constant and does not have boundary conditions
model.setInitAmount('Oral_dose', 0)
model.setConstant('Oral_dose', False)
model.setBoundary('Oral_dose', False)

model.setInitAmount('Drink_dose', 0)
model.setConstant('Drink_dose', False)
model.setBoundary('Drink_dose', False)

model.setInitAmount('Oral_on', 0)
model.setConstant('Oral_on', False)
model.setBoundary('Oral_on', False)

model.setInitAmount('Dermal_dose', 0)
model.setConstant('Dermal_dose', False)
model.setBoundary('Dermal_dose', False)

model.setInitAmount('Dermal_on', 0)
model.setConstant('Dermal_on', False)
model.setBoundary('Dermal_on', False)

Oraldose = 0.000187 # ug/kg
Drinkdose = 0.01 # ug/L, replaces Drinkconc
oral_exposure_start = 365 # 
oral_exposure_stop = 50*365

Dermaldose = 100 # ug/L
dermal_exposure_start = 15*365
dermal_exposure_stop = 35*365

num_days = 80*365
tsteps = (num_days*1)+1 # starting at 0 + 500 days = 501 timesteps

# Oral exposure to contaminated food
model.addEvent("Oral_exposure_dose", False, f"time == 1", False) # {oral_exposure_start}", False)
model.addEventAssignment("Oral_exposure_dose", "Oral_dose", f"{"Oral_dose"} + {Oraldose}", False)
# Oral exposure to contaminated drinking water
model.addEvent("Drink_exposure_dose", False, f"time == 1", False) # {oral_exposure_start}", False)
model.addEventAssignment("Drink_exposure_dose", "Drink_dose", f"{"Drink_dose"} + {Drinkdose}", False)
# Turning oral exposure on and off
model.addEvent("Oral_exposure_start", False, f"time == {oral_exposure_start}", False)
model.addEventAssignment("Oral_exposure_start", "Oral_on", f"{"Oral_on"} + 1", False)
model.addEvent("Oral_exposure_stop", False, f"time == {oral_exposure_stop}", False)
model.addEventAssignment("Oral_exposure_stop", "Oral_on", f"{"Oral_on"} - 1", False)

# Dermal exposure
model.addEvent("Dermal_exposure_dose", False, f"time == 1", False) # {dermal_exposure_start}", False)
model.addEventAssignment("Dermal_exposure_dose", "Dermal_dose", f"{"Dermal_dose"} + {Dermaldose}", False)
# Turning dermal exposure on and off
model.addEvent("Dermal_exposure_start", False, f"time == {dermal_exposure_start}", False)
model.addEventAssignment("Dermal_exposure_start", "Dermal_on", f"{"Dermal_on"} + 1", False)
model.addEvent("Dermal_exposure_stop", False, f"time == {dermal_exposure_stop}", False)
model.addEventAssignment("Dermal_exposure_stop", "Dermal_on", f"{"Dermal_on"} - 1", False)

model.regenerateModel(True, True)

# Simulate the PBPK model
plot_params = ['age','BW','Asc','Atrans','Ave','ASk','CLsc','AG','CG', 'AL','Aven','Vven_plas','Cven','Cart','Afil','Adelay','Aurine','Oral_dose','Drink_dose','Oral_on','Dermal_dose','Dermal_on']
results = model.simulate(0, num_days, tsteps, plot_params)
# Plot the results
plotResults(results,plot_params)